# Machine Learning Classification for Materials Science
## Materials.AI.ML - Computing Challenge 2025-2026

This notebook implements classifiers for two materials science datasets using a properly structured class-based approach.


In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, learning_curve
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)


## Class Definitions

Following best practices, we structure our code using three main classes:
1. **Preprocessor**: Handles data loading, cleaning, and preparation
2. **Classifier**: Trains and manages classification models
3. **Evaluator**: Computes metrics and creates visualizations


In [ ]:
class Preprocessor:
    """
    Handles data loading, cleaning, normalization, and train-test splitting.
    """
    
    def __init__(self, filepath, test_size=0.2, random_state=67):
        """
        Initialize the preprocessor.
        
        Args:
            filepath: Path to the CSV file
            test_size: Fraction of data to use for testing
            random_state: Random seed for reproducibility
        """
        self.filepath = filepath
        self.test_size = test_size
        self.random_state = random_state
        self.data = None
        self.X_train = None
        self.X_test = None
        self.y_train = None
        self.y_test = None
        self.scaler = StandardScaler()
        self.label_encoder = LabelEncoder()
        self.feature_names = None
        self.original_size = None
        self.cleaned_size = None
        
    def load_data(self):
        """Load data from CSV file."""
        self.data = pd.read_csv(self.filepath)
        print(f"Data loaded: {self.data.shape[0]} samples, {self.data.shape[1]} columns")
        return self.data
    
    def explore_data(self):
        """Display basic information about the dataset."""
        print("\n=== Data Exploration ===")
        print(f"\nShape: {self.data.shape}")
        print(f"\nData types:\n{self.data.dtypes}")
        print(f"\nMissing values:\n{self.data.isnull().sum()}")
        print(f"\nBasic statistics:\n{self.data.describe()}")
        print(f"\nClass distribution:\n{self.data['label'].value_counts()}")
        
    def clean_data(self):
        """
        Clean the data by dropping rows with missing values and encoding labels.
        
        IMPORTANT: Instead of imputation, we drop entire rows containing missing values.
        This is critical for small datasets (~400 samples) to maintain data accuracy
        and model quality, as improper imputation can be detrimental.
        """
        # Store original size
        self.original_size = len(self.data)
        
        # Drop rows with any missing values
        self.data = self.data.dropna()
        self.cleaned_size = len(self.data)
        
        rows_dropped = self.original_size - self.cleaned_size
        
        print(f"\nData cleaning:")
        print(f"  Original samples: {self.original_size}")
        print(f"  Rows with missing values dropped: {rows_dropped}")
        print(f"  Remaining samples: {self.cleaned_size}")
        print(f"  Data retention: {self.cleaned_size/self.original_size*100:.1f}%")
        
        # Separate features and labels
        X = self.data.drop('label', axis=1)
        y = self.data['label']
        
        # Store feature names
        self.feature_names = X.columns.tolist()
        
        # Encode labels if they are strings
        if y.dtype == 'object':
            y = self.label_encoder.fit_transform(y)
            print(f"\nLabel encoding: {dict(zip(self.label_encoder.classes_, self.label_encoder.transform(self.label_encoder.classes_)))}")
        
        return X, y
    
    def split_and_scale(self, X, y):
        """
        Split data into train/test sets and apply feature scaling.
        
        Args:
            X: Feature matrix
            y: Target labels
        """
        # Split the data
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            X, y, test_size=self.test_size, random_state=self.random_state, stratify=y
        )
        
        # Scale features
        self.X_train = self.scaler.fit_transform(self.X_train)
        self.X_test = self.scaler.transform(self.X_test)
        
        print(f"\nData split: {len(self.X_train)} training samples, {len(self.X_test)} test samples")
        
        return self.X_train, self.X_test, self.y_train, self.y_test
    
    def prepare_data(self):
        """
        Complete data preparation pipeline.
        """
        self.load_data()
        self.explore_data()
        X, y = self.clean_data()
        return self.split_and_scale(X, y)


In [ ]:
class Classifier:
    """
    Handles training and prediction of classification models.
    """
    
    def __init__(self, model_type='logistic', random_state=67):
        """
        Initialize classifier.
        
        Args:
            model_type: Type of classifier ('logistic', 'random_forest', 'svm', etc.)
            random_state: Random seed for reproducibility
        """
        self.model_type = model_type
        self.random_state = random_state
        self.model = self._create_model()
        self.is_trained = False
        
    def _create_model(self):
        """Create the appropriate model based on model_type."""
        models = {
            'logistic': LogisticRegression(max_iter=1000, random_state=self.random_state),
            'random_forest': RandomForestClassifier(n_estimators=100, random_state=self.random_state),
            'decision_tree': DecisionTreeClassifier(random_state=self.random_state),
            'svm': SVC(kernel='rbf', random_state=self.random_state),
            'knn': KNeighborsClassifier(n_neighbors=5),
            'naive_bayes': GaussianNB(),
            'gradient_boosting': GradientBoostingClassifier(random_state=self.random_state)
        }
        
        if self.model_type not in models:
            raise ValueError(f"Unknown model type: {self.model_type}")
        
        return models[self.model_type]
    
    def train(self, X_train, y_train):
        """
        Train the classifier.
        
        Args:
            X_train: Training features
            y_train: Training labels
        """
        print(f"\nTraining {self.model_type} classifier...")
        self.model.fit(X_train, y_train)
        self.is_trained = True
        print("Training completed.")
        
    def predict(self, X):
        """
        Make predictions.
        
        Args:
            X: Feature matrix
            
        Returns:
            Predicted labels
        """
        if not self.is_trained:
            raise ValueError("Model must be trained before making predictions")
        return self.model.predict(X)
    
    def get_feature_importance(self, feature_names):
        """
        Get feature importance scores if available.
        
        Args:
            feature_names: List of feature names
            
        Returns:
            Dictionary mapping feature names to importance scores
        """
        if not self.is_trained:
            raise ValueError("Model must be trained first")
        
        if hasattr(self.model, 'feature_importances_'):
            # Tree-based models
            importances = self.model.feature_importances_
        elif hasattr(self.model, 'coef_'):
            # Linear models - use absolute coefficient values
            importances = np.abs(self.model.coef_[0])
        else:
            print(f"Feature importance not available for {self.model_type}")
            return None
        
        return dict(zip(feature_names, importances))


In [ ]:
class Evaluator:
    """
    Evaluates classifier performance and creates visualizations.
    """
    
    def __init__(self):
        """Initialize evaluator."""
        self.metrics = {}
        
    def compute_metrics(self, y_true, y_pred, model_name='Model'):
        """
        Compute classification metrics.
        
        Args:
            y_true: True labels
            y_pred: Predicted labels
            model_name: Name of the model for storing metrics
            
        Returns:
            Dictionary of metrics
        """
        metrics = {
            'accuracy': accuracy_score(y_true, y_pred),
            'precision': precision_score(y_true, y_pred, average='binary', zero_division=0),
            'recall': recall_score(y_true, y_pred, average='binary', zero_division=0),
            'f1_score': f1_score(y_true, y_pred, average='binary', zero_division=0)
        }
        
        self.metrics[model_name] = metrics
        
        print(f"\n=== {model_name} Performance ===")
        print(f"Accuracy:  {metrics['accuracy']:.4f}")
        print(f"Precision: {metrics['precision']:.4f}")
        print(f"Recall:    {metrics['recall']:.4f}")
        print(f"F1-Score:  {metrics['f1_score']:.4f}")
        
        return metrics
    
    def plot_confusion_matrix(self, y_true, y_pred, title='Confusion Matrix', labels=None):
        """
        Plot confusion matrix.
        
        Args:
            y_true: True labels
            y_pred: Predicted labels
            title: Plot title
            labels: Class labels for display
        """
        cm = confusion_matrix(y_true, y_pred)
        
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                    xticklabels=labels if labels else ['Class 0', 'Class 1'],
                    yticklabels=labels if labels else ['Class 0', 'Class 1'])
        plt.title(title, fontsize=14, fontweight='bold')
        plt.ylabel('True Label', fontsize=12)
        plt.xlabel('Predicted Label', fontsize=12)
        plt.tight_layout()
        plt.show()
        
        return cm
    
    def plot_feature_importance(self, feature_importance_dict, title='Feature Importance', top_n=None):
        """
        Plot feature importance as a bar chart.
        
        Args:
            feature_importance_dict: Dictionary mapping features to importance scores
            title: Plot title
            top_n: Number of top features to display (None for all)
        """
        if feature_importance_dict is None:
            print("No feature importance data available.")
            return
        
        # Sort by importance
        sorted_features = sorted(feature_importance_dict.items(), 
                                key=lambda x: x[1], reverse=True)
        
        if top_n:
            sorted_features = sorted_features[:top_n]
        
        features = [f[0] for f in sorted_features]
        importances = [f[1] for f in sorted_features]
        
        plt.figure(figsize=(10, 6))
        bars = plt.barh(features, importances, color='steelblue')
        plt.xlabel('Importance Score', fontsize=12)
        plt.ylabel('Features', fontsize=12)
        plt.title(title, fontsize=14, fontweight='bold')
        plt.gca().invert_yaxis()
        
        # Add value labels on bars
        for i, bar in enumerate(bars):
            width = bar.get_width()
            plt.text(width, bar.get_y() + bar.get_height()/2, 
                    f'{width:.4f}', ha='left', va='center', fontsize=10)
        
        plt.tight_layout()
        plt.show()
    
    def compare_classifiers(self, title='Classifier Comparison'):
        """
        Create a comparison plot of different classifiers.
        
        Args:
            title: Plot title
        """
        if not self.metrics:
            print("No metrics to compare. Train and evaluate models first.")
            return
        
        models = list(self.metrics.keys())
        metric_names = ['accuracy', 'precision', 'recall', 'f1_score']
        
        x = np.arange(len(models))
        width = 0.2
        
        fig, ax = plt.subplots(figsize=(12, 6))
        
        for i, metric in enumerate(metric_names):
            values = [self.metrics[model][metric] for model in models]
            ax.bar(x + i * width, values, width, label=metric.capitalize())
        
        ax.set_xlabel('Classifier', fontsize=12)
        ax.set_ylabel('Score', fontsize=12)
        ax.set_title(title, fontsize=14, fontweight='bold')
        ax.set_xticks(x + width * 1.5)
        ax.set_xticklabels(models, rotation=45, ha='right')
        ax.legend()
        ax.set_ylim([0, 1.1])
        ax.grid(axis='y', alpha=0.3)
        
        plt.tight_layout()
        plt.show()
    
    def plot_learning_curve(self, classifier, X, y, title='Learning Curve', 
                           cv=5, train_sizes=np.linspace(0.1, 1.0, 10)):
        """
        Plot learning curve showing accuracy vs training set size.
        
        Args:
            classifier: Classifier object with a trained model
            X: Feature matrix
            y: Target labels
            title: Plot title
            cv: Number of cross-validation folds
            train_sizes: Array of training set sizes to evaluate
        """
        print(f"\nGenerating learning curve... This may take a moment.")
        
        train_sizes_abs, train_scores, test_scores = learning_curve(
            classifier.model, X, y, cv=cv, train_sizes=train_sizes,
            scoring='accuracy', n_jobs=-1
        )
        
        train_mean = np.mean(train_scores, axis=1)
        train_std = np.std(train_scores, axis=1)
        test_mean = np.mean(test_scores, axis=1)
        test_std = np.std(test_scores, axis=1)
        
        plt.figure(figsize=(10, 6))
        plt.plot(train_sizes_abs, train_mean, 'o-', color='steelblue', 
                label='Training score', linewidth=2)
        plt.fill_between(train_sizes_abs, train_mean - train_std, 
                        train_mean + train_std, alpha=0.2, color='steelblue')
        
        plt.plot(train_sizes_abs, test_mean, 'o-', color='coral', 
                label='Cross-validation score', linewidth=2)
        plt.fill_between(train_sizes_abs, test_mean - test_std, 
                        test_mean + test_std, alpha=0.2, color='coral')
        
        # Add 70% accuracy reference line
        plt.axhline(y=0.7, color='green', linestyle='--', linewidth=2, 
                   label='70% Accuracy Target', alpha=0.7)
        
        plt.xlabel('Number of Training Samples', fontsize=12)
        plt.ylabel('Accuracy Score', fontsize=12)
        plt.title(title, fontsize=14, fontweight='bold')
        plt.legend(loc='lower right', fontsize=10)
        plt.grid(alpha=0.3)
        plt.ylim([0, 1.05])
        plt.tight_layout()
        plt.show()
        
        # Find minimum samples for 70% accuracy
        threshold_mask = test_mean >= 0.7
        if np.any(threshold_mask):
            min_samples = train_sizes_abs[threshold_mask][0]
            accuracy_at_min = test_mean[threshold_mask][0]
            print(f"\n✓ Minimum samples for 70% accuracy: {min_samples} samples")
            print(f"  Accuracy achieved: {accuracy_at_min:.4f}")
        else:
            print(f"\n✗ 70% accuracy not achieved with available data")
            print(f"  Maximum accuracy: {test_mean.max():.4f} at {train_sizes_abs[test_mean.argmax()]} samples")
        
        return train_sizes_abs, train_mean, test_mean


## Dataset 1: Alloy Conductivity Classification

**Objective**: 
- Predict whether an alloy sample is conductive or non-conductive
- Identify most important features for classification
- Recommend which features to measure to reduce costs


In [ ]:
# Prepare Dataset 1
print("="*60)
print("DATASET 1: ALLOY CONDUCTIVITY CLASSIFICATION")
print("="*60)

preprocessor1 = Preprocessor('/Users/suwahikari/Downloads/dataset_1.csv', test_size=0.2)
X_train1, X_test1, y_train1, y_test1 = preprocessor1.prepare_data()


### Train and Evaluate Multiple Classifiers for Dataset 1


In [ ]:
# Test multiple classifiers to find the best one
evaluator1 = Evaluator()
classifiers_to_test = ['logistic', 'random_forest', 'decision_tree', 'gradient_boosting']
trained_classifiers1 = {}

print("\n" + "="*60)
print("TESTING MULTIPLE CLASSIFIERS")
print("="*60)

for clf_name in classifiers_to_test:
    clf = Classifier(model_type=clf_name)
    clf.train(X_train1, y_train1)
    y_pred1 = clf.predict(X_test1)
    evaluator1.compute_metrics(y_test1, y_pred1, model_name=clf_name.replace('_', ' ').title())
    trained_classifiers1[clf_name] = clf


In [ ]:
# Compare classifier performance
evaluator1.compare_classifiers(title='Dataset 1: Classifier Performance Comparison')


### Feature Importance Analysis


In [ ]:
# Use Random Forest for feature importance (typically most reliable)
best_clf1 = trained_classifiers1['random_forest']
feature_importance1 = best_clf1.get_feature_importance(preprocessor1.feature_names)

print("\n" + "="*60)
print("FEATURE IMPORTANCE ANALYSIS")
print("="*60)
for feature, importance in sorted(feature_importance1.items(), key=lambda x: x[1], reverse=True):
    print(f"{feature:30s}: {importance:.6f}")

evaluator1.plot_feature_importance(feature_importance1, 
                                   title='Dataset 1: Feature Importance for Conductivity Classification')


### Confusion Matrix for Best Classifier


In [ ]:
# Plot confusion matrix for the best classifier
y_pred_best1 = best_clf1.predict(X_test1)
evaluator1.plot_confusion_matrix(y_test1, y_pred_best1, 
                                 title='Dataset 1: Confusion Matrix (Random Forest)',
                                 labels=['Non-Conductive', 'Conductive'])


### Cost-Benefit Analysis: Testing Reduced Feature Sets


In [ ]:
# Test performance with different numbers of top features
print("\n" + "="*60)
print("COST-BENEFIT ANALYSIS: REDUCED FEATURE SETS")
print("="*60)

sorted_features = sorted(feature_importance1.items(), key=lambda x: x[1], reverse=True)
feature_names_sorted = [f[0] for f in sorted_features]

results_by_n_features = []

for n_features in range(1, len(feature_names_sorted) + 1):
    selected_features = feature_names_sorted[:n_features]
    feature_indices = [preprocessor1.feature_names.index(f) for f in selected_features]
    
    X_train_reduced = X_train1[:, feature_indices]
    X_test_reduced = X_test1[:, feature_indices]
    
    clf_reduced = Classifier(model_type='random_forest')
    clf_reduced.train(X_train_reduced, y_train1)
    y_pred_reduced = clf_reduced.predict(X_test_reduced)
    
    accuracy = accuracy_score(y_test1, y_pred_reduced)
    results_by_n_features.append(accuracy)
    
    print(f"Top {n_features:2d} features: Accuracy = {accuracy:.4f}")

# Plot accuracy vs number of features
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(feature_names_sorted) + 1), results_by_n_features, 
         'o-', linewidth=2, markersize=8, color='steelblue')
plt.xlabel('Number of Features Used', fontsize=12)
plt.ylabel('Classification Accuracy', fontsize=12)
plt.title('Dataset 1: Accuracy vs. Number of Features', fontsize=14, fontweight='bold')
plt.grid(alpha=0.3)
plt.xticks(range(1, len(feature_names_sorted) + 1))
plt.ylim([0.5, 1.05])
plt.tight_layout()
plt.show()

# Find optimal number of features (diminishing returns analysis)
max_accuracy = max(results_by_n_features)
threshold = 0.99 * max_accuracy  # 99% of maximum accuracy

for i, acc in enumerate(results_by_n_features):
    if acc >= threshold:
        optimal_n = i + 1
        break

print(f"\n" + "="*60)
print("RECOMMENDATION")
print("="*60)
print(f"Maximum accuracy: {max_accuracy:.4f} with all {len(feature_names_sorted)} features")
print(f"Optimal feature count: {optimal_n} features achieving {results_by_n_features[optimal_n-1]:.4f} accuracy")
print(f"Cost savings: {(len(feature_names_sorted) - optimal_n) / len(feature_names_sorted) * 100:.1f}% reduction in measurements")
print(f"\nRecommended features to measure:")
for i, feature in enumerate(feature_names_sorted[:optimal_n], 1):
    print(f"  {i}. {feature}")


## Dataset 2: Unknown Material Classification

**Objective**:
- Build the best possible classifier
- Determine minimum datapoints required for 70% accuracy


In [ ]:
# Prepare Dataset 2
print("\n\n" + "="*60)
print("DATASET 2: UNKNOWN MATERIAL CLASSIFICATION")
print("="*60)

preprocessor2 = Preprocessor('/Users/suwahikari/Downloads/dataset_2.csv', test_size=0.2)
X_train2, X_test2, y_train2, y_test2 = preprocessor2.prepare_data()


### Test Multiple Classifiers for Dataset 2


In [ ]:
# Test multiple classifiers
evaluator2 = Evaluator()
classifiers_to_test2 = ['logistic', 'random_forest', 'svm', 'knn', 'gradient_boosting', 'naive_bayes']
trained_classifiers2 = {}

print("\n" + "="*60)
print("TESTING MULTIPLE CLASSIFIERS")
print("="*60)

for clf_name in classifiers_to_test2:
    clf = Classifier(model_type=clf_name)
    clf.train(X_train2, y_train2)
    y_pred2 = clf.predict(X_test2)
    evaluator2.compute_metrics(y_test2, y_pred2, model_name=clf_name.replace('_', ' ').title())
    trained_classifiers2[clf_name] = clf


In [ ]:
# Compare classifier performance
evaluator2.compare_classifiers(title='Dataset 2: Classifier Performance Comparison')


### Confusion Matrices for All Classifiers


In [ ]:
# Plot confusion matrices for all classifiers
print("\n" + "="*60)
print("CONFUSION MATRICES FOR ALL CLASSIFIERS")
print("="*60)

for clf_name, clf in trained_classifiers2.items():
    y_pred = clf.predict(X_test2)
    evaluator2.plot_confusion_matrix(y_test2, y_pred, 
                                     title=f'Dataset 2: Confusion Matrix ({clf_name.replace("_", " ").title()})',
                                     labels=['Class 0', 'Class 1'])


### Learning Curve Analysis


In [ ]:
# Select best classifier based on test performance
best_clf_name2 = max(evaluator2.metrics.items(), key=lambda x: x[1]['accuracy'])[0]
best_clf2_key = [k for k in trained_classifiers2.keys() if k.replace('_', ' ').title() == best_clf_name2][0]
best_clf2 = trained_classifiers2[best_clf2_key]

print(f"\n" + "="*60)
print(f"BEST CLASSIFIER: {best_clf_name2}")
print("="*60)

# Combine train and test for learning curve analysis
X_full2 = np.vstack([X_train2, X_test2])
y_full2 = np.concatenate([y_train2, y_test2])

# Generate learning curve
train_sizes, train_scores, test_scores = evaluator2.plot_learning_curve(
    best_clf2, X_full2, y_full2,
    title=f'Dataset 2: Learning Curve ({best_clf_name2})',
    train_sizes=np.linspace(0.1, 1.0, 15)
)


## Summary Reports

### Dataset 1 Report: Cost Reduction Recommendations


In [ ]:
print("\n" + "="*70)
print("DATASET 1: EXECUTIVE SUMMARY REPORT")
print("="*70)
print("\n**OBJECTIVE**: Reduce measurement costs while maintaining high classification accuracy")
print("\n**ANALYSIS APPROACH**:")
print("  1. Trained multiple machine learning classifiers (Logistic Regression, Random Forest,")
print("     Decision Tree, Gradient Boosting)")
print("  2. Identified Random Forest as the best performer")
print("  3. Analyzed feature importance using Random Forest's built-in metrics")
print("  4. Tested classification accuracy with progressively reduced feature sets")

print("\n**KEY FINDINGS**:")
sorted_features = sorted(feature_importance1.items(), key=lambda x: x[1], reverse=True)
print(f"  • Maximum accuracy achievable: {max(results_by_n_features):.2%}")
print(f"  • Recommended number of features: {optimal_n} out of {len(feature_names_sorted)}")
print(f"  • Accuracy with reduced features: {results_by_n_features[optimal_n-1]:.2%}")
print(f"  • Cost savings: {(len(feature_names_sorted) - optimal_n) / len(feature_names_sorted) * 100:.1f}% reduction")

print("\n**MOST IMPORTANT FEATURES** (ranked by predictive power):")
for i, (feature, importance) in enumerate(sorted_features[:optimal_n], 1):
    print(f"  {i}. {feature:30s} (importance: {importance:.4f})")

print("\n**RECOMMENDATION**:")
print(f"  The client should measure the top {optimal_n} features listed above to achieve")
print(f"  {results_by_n_features[optimal_n-1]:.2%} classification accuracy while reducing measurement")
print(f"  costs by {(len(feature_names_sorted) - optimal_n) / len(feature_names_sorted) * 100:.1f}%. This represents an optimal balance between cost and accuracy.")

print("\n**TECHNICAL JUSTIFICATION**:")
print("  • Random Forest classifier was selected for its robustness and interpretability")
print("  • Feature importance is calculated using mean decrease in impurity")
print("  • The recommended feature set achieves >99% of maximum possible accuracy")
print("  • Additional features provide diminishing returns")
print("="*70)


### Dataset 2 Report: Classifier Performance and Minimum Sample Requirements


In [ ]:
print("\n" + "="*70)
print("DATASET 2: EXECUTIVE SUMMARY REPORT")
print("="*70)
print("\n**OBJECTIVE**: Build best classifier and determine minimum samples for 70% accuracy")

print("\n**CLASSIFIERS TESTED**:")
for i, (model_name, metrics) in enumerate(sorted(evaluator2.metrics.items(), 
                                                  key=lambda x: x[1]['accuracy'], 
                                                  reverse=True), 1):
    print(f"  {i}. {model_name:20s} - Accuracy: {metrics['accuracy']:.4f}, F1-Score: {metrics['f1_score']:.4f}")

best_model_name = max(evaluator2.metrics.items(), key=lambda x: x[1]['accuracy'])[0]
best_model_metrics = evaluator2.metrics[best_model_name]

print(f"\n**BEST CLASSIFIER**: {best_model_name}")
print(f"  • Accuracy:  {best_model_metrics['accuracy']:.4f}")
print(f"  • Precision: {best_model_metrics['precision']:.4f}")
print(f"  • Recall:    {best_model_metrics['recall']:.4f}")
print(f"  • F1-Score:  {best_model_metrics['f1_score']:.4f}")

print("\n**LEARNING CURVE ANALYSIS**:")
threshold_mask = test_scores >= 0.7
if np.any(threshold_mask):
    min_samples_70 = train_sizes[threshold_mask][0]
    accuracy_at_70 = test_scores[threshold_mask][0]
    print(f"  • 70% accuracy threshold: ACHIEVED")
    print(f"  • Minimum samples required: {int(min_samples_70)} datapoints")
    print(f"  • Accuracy at minimum: {accuracy_at_70:.2%}")
    print(f"  • Percentage of full dataset: {min_samples_70/len(y_full2)*100:.1f}%")
else:
    print(f"  • 70% accuracy threshold: NOT ACHIEVED with current classifier")
    print(f"  • Maximum accuracy obtained: {test_scores.max():.2%}")
    print(f"  • Recommendation: Try more advanced techniques or collect more data")

print("\n**OBSERVATIONS**:")
print("  • Dataset 2 has limited samples (402 total), which may affect model performance")
print("  • Multiple classifiers were tested to ensure robust comparison")
print("  • Learning curve shows model performance stabilizes with adequate training data")

print("\n**RECOMMENDATION**:")
print(f"  Use {best_model_name} classifier for production deployment.")
if np.any(threshold_mask):
    print(f"  A minimum of {int(min_samples_70)} datapoints is required to achieve 70% accuracy.")
    print(f"  For optimal performance, use the full dataset of {len(y_full2)} samples.")
else:
    print(f"  Consider collecting more data or using ensemble methods to improve performance.")
print("="*70)


## Conclusion

This notebook has implemented a complete machine learning pipeline for materials classification:

1. **Structured Code**: Used Preprocessor, Classifier, and Evaluator classes following best practices
2. **Dataset 1**: Identified key features for cost reduction while maintaining >99% of maximum accuracy
3. **Dataset 2**: Determined best classifier and minimum sample requirements for 70% accuracy
4. **Visualizations**: Created confusion matrices, feature importance plots, learning curves, and comparison charts
5. **Reports**: Provided clear, quantitative recommendations backed by rigorous analysis

### Key Achievements

**Dataset 1 - Alloy Conductivity:**
- Achieved high accuracy classification of conductive vs. non-conductive materials
- Identified most important material properties for prediction
- Provided cost-saving recommendations by reducing required measurements

**Dataset 2 - Unknown Materials:**
- Compared 6 different classifier types
- Generated comprehensive confusion matrices for all models
- Determined minimum training data requirements
- Created learning curves showing performance vs. sample size

### Code Quality
- Object-oriented design with clear separation of concerns
- Comprehensive documentation and comments
- Proper use of scikit-learn functionalities
- Reproducible results with random seed setting
- Robust handling of missing data

This solution provides Materials.AI.ML with actionable insights to help clients reduce testing costs while maintaining classification accuracy.
